In [1]:
from AlgoTools import AlgoTools
from Trader import Trader
import pandas as pd
import time
import datetime
import pickle
import numpy as np

# Processing the data
- The data has already been grouped into days and converted to date time objects
- Objective is to shuffle the days, then create a train-test split as I will be training on indivuals days

In [2]:
with open('SP-groupedByDays.pkl', 'rb') as f:
    spDaily = pickle.load(f)

In [3]:
with open('spydf.pickle', 'rb') as f:
    tickData = pickle.load(f)

In [5]:
trainLen = int(len(spDaily)*0.75)

In [4]:
import random
random.shuffle(spDaily)

In [6]:
trainData=spDaily[:trainLen]
testData=spDaily[trainLen:]

In [11]:
tempData=spDaily[:10]

# Creating the Algorithim

In [12]:
algoTools = AlgoTools(tickData)

In [13]:
def Algo(data,r, a_L, a_S, E_e, stopProf, stopLoss):
    trader = Trader()
    #for each day in the data
    for d in range(len(data)):
        #for each time stamp in the day
        index = 1
        
        for t in range(data[d].index[2] , data[d].index[len(data[d])-1]):
            index = index + 1
            
            #close out because market is closing                
            ti = tickData.loc[t,"Time2"].hour*100 + tickData.loc[t,"Time2"].minute
            if (1513<ti<1520 or 2330<ti<2359):
                if trader.getLastTradePos() != 0:
                    trader.closeOut(data[d].iloc[index,:])
                continue
            
            


            
            #if the acceleration is greater than or equal to acceleration constant Long
            if (algoTools.getAccel(t) >= a_L or algoTools.getRate(t) >= r) and trader.getLastTradePos() == 0:
                trader.goLong(data[d].iloc[index,:], "A_L reached")
                continue
            
            #same as previous statment but for shorting
            if  ( algoTools.getAccel(t) <= a_S or algoTools.getRate(t) <= -r ) and trader.getLastTradePos() == 0:
                trader.goShort(data[d].iloc[index,:],"A_S reached")
                continue
            
            
            
            
           
            
            #-----Closing Conditions------
            
            
            #as per calclus reasoing if the first deravitive is = 0 it has found a local extrema. therfore close out 
            #E_e is error of extrema
            if algoTools.getRate(t) <= E_e and trader.getLastTradePos() != 0:
                trader.closeOut(data[d].iloc[index,:], "E_e reached")
                continue
               
            
            #stop profit (in points)
            if (trader.getLastTradePos() == 1 and data[d].loc[t,"price"] - trader.getLastTradePrice() >= stopProf ) or (trader.getLastTradePos() == -1 and trader.getLastTradePrice() - data[d].loc[t,"price"] >= stopProf):
                trader.closeOut(data[d].iloc[index,:],"Stop Prof Reached")
                continue
            
            #stop loss (in points)
            if (trader.getLastTradePos() == 1 and data[d].loc[t,"price"] - trader.getLastTradePrice() <= stopLoss) or (trader.getLastTradePos() == -1 and trader.getLastTradePrice() - data[d].loc[t,"price"] <= stopLoss ):
                trader.closeOut(data[d].iloc[index,:],"Stop Loss Reached")
                continue
                
            
            
                
            
    return trader

In [14]:
trader = Algo(tempData,1,0.5,-0.5,0.01,2,0.5)

In [15]:
trader.getTradeInfo()

{'Total Return': 6220.000000000124, 'Max Return': 100.0, 'Min Return': -50.0}

In [16]:
trader.getLR()

0    0.335751
dtype: float64

In [13]:
trader.getTradeData()

,Position,TradePrice,TradeTime,Method
0,-1.0,1219.8,2005-08-23 15:30:00,A_S reached
1,0.0,1219.8,2005-08-23 15:30:00,E_e reached
2,1.0,1221.3,2005-08-23 15:30:00,A_L reached
3,0.0,1221.3,2005-08-23 15:30:00,E_e reached
4,-1.0,1220.4,2005-08-23 17:00:00,A_S reached
...,...,...,...,...
2059,0.0,1176.6,2010-10-14 07:30:03,E_e reached
2060,1.0,1175.3,2010-10-14 07:30:11,A_L reached
2061,0.0,1175.2,2010-10-14 07:30:11,E_e reached
2062,1.0,1176.2,2010-10-14 07:30:23,A_L reached


In [ ]:
from scipy.optimize import minimize



# Initial guess for the parameters
initial_guess = [1,0.5,-0.5,0.01,2,0.5]

# Perform the minimization
result = minimize(Algo, initial_guess)

# Output the result
print("Optimized parameters:", result.x)
print("Function value at optimized parameters:", result.fun)